In [97]:
import os
from openai import OpenAI
import time
import json
from bigbench.api import json_task
from tqdm import tqdm
import random

# Set up OpenAI client using environment variable for API key
client = OpenAI(
    api_key = 'LA-b66744d449e94d7db2496843d9044308604a57fc8a0b42019c2ddac89ef8052d',
    base_url="https://api.llama-api.com"
)

# Different scales of LLaMA model variants to evaluate
model_variants = [
    'llama3.2-90b-vision',
    'llama3.2-11b-vision',
    'llama3.2-3b',
    'llama3.2-1b',
    'mixtral-8x22b-instruct',
    'mixtral-8x7b-instruct',
    'mistral-7b-instruct',
]

# Function to evaluate a specific model on the BIG-bench dataset
def evaluate_model(model_name, dataset, num_exp=50):
    # print(f"Evaluating model: {model_name}")
    results = []
    if 'examples' not in dataset:
        raise KeyError("The dataset does not contain the expected 'examples' key.")
    examples = random.sample(dataset['examples'], min(num_exp, len(dataset['examples'])))
    for example in tqdm(examples, desc=f"{dataset['name']}"):
        prompt = example['input']
        target = example['target']

        retry_count = 0
        while retry_count < 3:
            try:
                # Generate response using OpenAI client
                response = client.chat.completions.create(
                    model=model_name,
                    messages=[
                        {
                            "role": "system",
                            "content": "Assistant is a large language model trained by OpenAI."
                        },
                        {
                            "role": "user",
                            "content": prompt,
                        }
                    ]
                )
                # print(response)
                try: 
                    generated_text = response.choices[0].message.content.strip()
                except:
                    generated_text = response[0]['message']['content'].strip() if isinstance(response, list) else response['choices'][0]['message']['content'].strip()
                # Store prompt, generated response, and target for analysis
                results.append({
                    'prompt': prompt,
                    'generated_text': generated_text,
                    'target': target
                })
                break
            except Exception as e:
                print(f"Error occurred for model {model_name}: {e}")
                retry_count += 1
                time.sleep(2)  # Wait before retrying
                if retry_count == 3:
                    print(f"Failed to get response after 3 attempts for prompt: {prompt}")
    return results

### Testing on simple_arithmetic

In [ ]:
# Load BIG-bench dataset - replace 'path/to/BIG-bench' with actual path to the JSON file
json_path = '/Users/shawna_song/Desktop/598tz_proj/BIG-bench/bigbench/benchmark_tasks/simple_arithmetic_json/task.json'

if not os.path.exists(json_path):
    raise FileNotFoundError(f"The specified path does not exist: {json_path}")

with open(json_path, 'r') as f:
    bigbench_dataset = json.load(f)

# Function to evaluate a specific model on the BIG-bench dataset
def evaluate_model(model_name, dataset):
    print(f"Evaluating model: {model_name}")
    results = []

    if 'examples' not in dataset:
        raise KeyError("The dataset does not contain the expected 'examples' key.")

    for example in dataset['examples']:
        prompt = example['input']
        target = example['target']

        retry_count = 0
        while retry_count < 3:
            try:
                # Generate response using OpenAI client
                response = client.chat.completions.create(
                    model=model_name,
                    messages=[
                        {
                            "role": "system",
                            "content": "Assistant is a large language model trained by OpenAI."
                        },
                        {
                            "role": "user",
                            "content": prompt,
                        }
                    ]
                )
                # print(response.choices[0])
                try: 
                    generated_text = response.choices[0].message.content.strip()
                except:
                    generated_text = response[0]['message']['content'].strip() if isinstance(response, list) else response['choices'][0]['message']['content'].strip()
                # Store prompt, generated response, and target for analysis
                results.append({
                    'prompt': prompt,
                    'generated_text': generated_text,
                    'target': target
                })
                break
            except Exception as e:
                print(f"Error occurred for model {model_name}: {e}")
                retry_count += 1
                time.sleep(2)  # Wait before retrying
                if retry_count == 3:
                    print(f"Failed to get response after 3 attempts for prompt: {prompt}")
    return results

# Evaluate each model variant and store results
all_results = {}
for model in model_variants:
    all_results[model] = evaluate_model(model, bigbench_dataset)

# # Print the results for analysis
# for model, results in all_results.items():
#     print(f"\nResults for {model}:")
#     for result in results:
#         print(f"Prompt: {result['prompt']}")
#         print(f"Generated: {result['generated_text']}")
#         print(f"Target: {result['target']}\n")

Evaluating model: llama3.2-90b-vision
Evaluating model: llama3.2-11b-vision
Evaluating model: llama3.2-3b
Evaluating model: llama3.2-1b
Evaluating model: mixtral-8x22b-instruct
Evaluating model: mixtral-8x7b-instruct
Evaluating model: mistral-7b-instruct


In [38]:
# Function to calculate accuracy of the model
def calculate_accuracy(results):
    correct_count = 0
    total_count = len(results)

    for result in results:
        # if result['prompt'] + " " + result['target'] in result['generated_text']:
        #     correct_count += 1
        # elif result['target'] == result['generated_text']:
        #     correct_count += 1
        if result['target'] in result['generated_text']:
            correct_count += 1

    accuracy = (correct_count / total_count) * 100 if total_count > 0 else 0
    return accuracy

# Calculate and print accuracy for each model
for model, results in all_results.items():
    accuracy = calculate_accuracy(results)
    print(f"Accuracy for {model}: {accuracy:.2f}%")


Accuracy for llama3.2-90b-vision: 100.00%
Accuracy for llama3.2-11b-vision: 100.00%
Accuracy for llama3.2-3b: 90.00%
Accuracy for llama3.2-1b: 90.00%
Accuracy for mixtral-8x22b-instruct: 100.00%
Accuracy for mixtral-8x7b-instruct: 100.00%
Accuracy for mistral-7b-instruct: 96.67%


### Testing on modified_arithmetic

In [ ]:
model_variants = [
    'llama3.2-90b-vision',
    'llama3.2-11b-vision',
    'llama3.2-3b',
    'llama3.2-1b',
    # 'mixtral-8x22b-instruct',
    # 'mixtral-8x7b-instruct',
    # 'mistral-7b-instruct',
    # 'llama3.1-405b',
    # 'llama3.1-70b',
    # 'llama3.1-8b
]

In [ ]:
# Function to evaluate a specific model on the BIG-bench dataset
def evaluate_model(model_name, dataset, num_exp=50):
    # print(f"Evaluating model: {model_name}")
    results = []
    if 'examples' not in dataset:
        raise KeyError("The dataset does not contain the expected 'examples' key.")
    examples = random.sample(dataset['examples'], min(num_exp, len(dataset['examples'])))
    for example in tqdm(examples, desc=f"{dataset['name']}"):
        prompt = example['input']
        target = example['target']

        retry_count = 0
        while retry_count < 3:
            try:
                # Generate response using OpenAI client
                response = client.chat.completions.create(
                    model=model_name,
                    messages=[
                        {
                            "role": "system",
                            "content": "Assistant is a large language model trained by OpenAI."
                        },
                        {
                            "role": "user",
                            "content": prompt,
                        }
                    ]
                )
                # print(response)
                try: 
                    generated_text = response.choices[0].message.content.strip()
                except:
                    generated_text = response[0]['message']['content'].strip() if isinstance(response, list) else response['choices'][0]['message']['content'].strip()
                # Store prompt, generated response, and target for analysis
                results.append({
                    'prompt': prompt,
                    'generated_text': generated_text,
                    'target': target
                })
                break
            except Exception as e:
                print(f"Error occurred for model {model_name}: {e}")
                retry_count += 1
                time.sleep(2)  # Wait before retrying
                if retry_count == 3:
                    print(f"Failed to get response after 3 attempts for prompt: {prompt}")
    return results

base_path = '/Users/shawna_song/Desktop/598tz_proj/BIG-bench/bigbench/benchmark_tasks/modified_arithmetic'

all_results = {}
for model in model_variants:
    print(f"Evaluating model: {model}")
    all_results[model] = []
    for task_name in os.listdir(base_path):
        task_path = os.path.join(base_path, task_name)
        task_json_path = os.path.join(task_path, 'task.json')

        if os.path.isdir(task_path) and os.path.exists(task_json_path):
            with open(task_json_path, 'r') as f:
                bigbench_dataset = json.load(f)
            # print(f"Dataset structure for {task_name}:", json.dumps(bigbench_dataset, indent=2))
            # Evaluate each model variant and store results
            all_results[model] += evaluate_model(model, bigbench_dataset)

# Print the results for analysis
# for model, results in all_results.items():
#     print(f"\nResults for {model}:")
#     for result in results:
#         print(f"Prompt: {result['prompt']}")
#         print(f"Generated: {result['generated_text']}")
#         print(f"Target: {result['target']}\n")

Evaluating model: llama3.2-90b-vision


two_digit_multiplication_control:  10%|█         | 5/50 [00:20<02:52,  3.84s/it]

Error occurred for model llama3.2-90b-vision: 'ChatCompletion' object is not subscriptable


two_digit_multiplication_control:  22%|██▏       | 11/50 [01:00<03:57,  6.10s/it]

Error occurred for model llama3.2-90b-vision: 'ChatCompletion' object is not subscriptable
Error occurred for model llama3.2-90b-vision: 'ChatCompletion' object is not subscriptable


...

three_digit_subtraction_plus_one:  86%|████████▌ | 43/50 [02:08<00:31,  4.46s/it]

Error occurred for model llama3.2-3b: 'ChatCompletion' object is not subscriptable


three_digit_subtraction_control: 100%|██████████| 50/50 [01:13<00:00,  1.48s/it]


Evaluating model: llama3.2-1b


three_digit_subtraction_control: 100%|██████████| 50/50 [01:03<00:00,  1.28s/it]


In [82]:
def calculate_accuracy(results):
    correct_count = 0
    total_count = len(results)

    for result in results:
        # if result['prompt'] + " " + result['target'] in result['generated_text']:
        #     correct_count += 1
        # elif result['target'] == result['generated_text']:
        #     correct_count += 1
        if result['target'] in result['generated_text']:
            correct_count += 1

    accuracy = (correct_count / total_count) * 100 if total_count > 0 else 0
    return accuracy

# Calculate and print accuracy for each model
for model, results in all_results.items():
    accuracy = calculate_accuracy(results)
    print(f"Accuracy for {model}: {accuracy:.2f}%")

Accuracy for llama3.2-90b-vision: 39.00%
Accuracy for llama3.2-11b-vision: 30.33%
Accuracy for llama3.2-3b: 28.00%
Accuracy for llama3.2-1b: 28.67%


### Testing on international_phonetic_alphabet_transliterate

In [83]:
model_variants = [
    'llama3.2-90b-vision',
    'llama3.2-11b-vision',
    'llama3.2-3b',
    'llama3.2-1b',
    # 'mixtral-8x22b-instruct',
    # 'mixtral-8x7b-instruct',
    # 'mistral-7b-instruct',
    # 'llama3.1-405b',
    # 'llama3.1-70b',
    # 'llama3.1-8b
]

task_json_path = '/Users/shawna_song/Desktop/598tz_proj/BIG-bench/bigbench/benchmark_tasks/international_phonetic_alphabet_transliterate/task.json'

all_results = {}
for model in model_variants:
    print(f"Evaluating model: {model}")
    all_results[model] = []
    if os.path.isdir(task_path) and os.path.exists(task_json_path):
        with open(task_json_path, 'r') as f:
            bigbench_dataset = json.load(f)
        # print(f"Dataset structure for {task_name}:", json.dumps(bigbench_dataset, indent=2))
        # Evaluate each model variant and store results
        all_results[model] += evaluate_model(model, bigbench_dataset)

Evaluating model: llama3.2-90b-vision


international_phonetic_alphabet_transliterate: 100%|██████████| 50/50 [02:02<00:00,  2.46s/it]


Evaluating model: llama3.2-11b-vision


international_phonetic_alphabet_transliterate: 100%|██████████| 50/50 [01:09<00:00,  1.39s/it]


Evaluating model: llama3.2-3b


international_phonetic_alphabet_transliterate: 100%|██████████| 50/50 [01:16<00:00,  1.53s/it]


Evaluating model: llama3.2-1b


international_phonetic_alphabet_transliterate: 100%|██████████| 50/50 [00:54<00:00,  1.10s/it]


In [84]:
# Print the results for analysis
for model, results in all_results.items():
    print(f"\nResults for {model}:")
    for result in results:
        print(f"Prompt: {result['prompt']}")
        print(f"Generated: {result['generated_text']}")
        print(f"Target: {result['target']}\n")


Results for llama3.2-90b-vision:
Prompt: IPA: ðə ˈɛntɪti ʃʊd nɑt bi sin, ˈoʊnli hərd.
English:
Generated: The entity should not be seen, only heard.
Target:  The entity should not be seen, only heard.

Prompt: IPA: ɔn hɪz ˈbərθˌdeɪ strɑm ˈθərmənd əˈnaʊnst hɪz rɪˈtaɪərmənt frəm ðə ˈsɛnɪt ɑrmd ˈsərvɪsɪz kəˈmɪti ɪn wən jɪr.
English:
Generated: On his birthday, he was summoned against his retirement from the Senate Armed Services Committee in one year.
Target:  On his Birthday Strom Thurmond announced his retirement from the Senate Armed Services Committee in one year.

Prompt: English: We can sleep all day, since we don't have any daytime feedings.
IPA:
Generated: /wi kən sliːp ɔːl deɪ, sɪns wi dəʊnt hæv ɛni deɪtaɪm fiːdɪŋz/
Target:  wi kən slip ɔl deɪ, sɪns wi doʊnt hæv ˈɛni ˈdeɪˌtaɪm ˈfidɪŋz.

Prompt: English: It is a terrible situation. 
IPA:
Generated: /ɪt ɪz ə tɛrəbəl sɪtʃuˈeɪʃən/
Target:  ɪt ɪz ə ˈtɛrəbəl ˌsɪʧuˈeɪʃən.

Prompt: IPA: ju nid tɪ pæs θru ɪt ɔn fʊt.
English:
Generated: Y

In [86]:
from nltk.translate.bleu_score import sentence_bleu

def calculate_bleu(results):
    total_bleu = 0
    total_count = len(results)

    for result in results:
        reference = [result['target'].split()]
        candidate = result['generated_text'].split()
        bleu_score = sentence_bleu(reference, candidate)
        total_bleu += bleu_score

    average_bleu = (total_bleu / total_count) * 100 if total_count > 0 else 0
    return average_bleu

# Calculate and print accuracy for each model
for model, results in all_results.items():
    accuracy = calculate_bleu(results)
    print(f"BLEU for {model}: {accuracy:.2f}%")

BLEU for llama3.2-90b-vision: 46.91%
BLEU for llama3.2-11b-vision: 9.67%
BLEU for llama3.2-3b: 9.79%
BLEU for llama3.2-1b: 2.92%


In [87]:
model_variants = [
    'mixtral-8x22b-instruct',
    'mixtral-8x7b-instruct',
    'mistral-7b-instruct',
]

task_json_path = '/Users/shawna_song/Desktop/598tz_proj/BIG-bench/bigbench/benchmark_tasks/international_phonetic_alphabet_transliterate/task.json'

all_results = {}
for model in model_variants:
    print(f"Evaluating model: {model}")
    all_results[model] = []
    if os.path.isdir(task_path) and os.path.exists(task_json_path):
        with open(task_json_path, 'r') as f:
            bigbench_dataset = json.load(f)
        # print(f"Dataset structure for {task_name}:", json.dumps(bigbench_dataset, indent=2))
        # Evaluate each model variant and store results
        all_results[model] += evaluate_model(model, bigbench_dataset)
        
for model, results in all_results.items():
    accuracy = calculate_bleu(results)
    print(f"BLEU for {model}: {accuracy:.2f}%")

Evaluating model: mixtral-8x22b-instruct


international_phonetic_alphabet_transliterate: 100%|██████████| 50/50 [01:59<00:00,  2.38s/it]


Evaluating model: mixtral-8x7b-instruct


international_phonetic_alphabet_transliterate: 100%|██████████| 50/50 [01:03<00:00,  1.26s/it]


Evaluating model: mistral-7b-instruct


international_phonetic_alphabet_transliterate:  10%|█         | 5/50 [00:08<01:09,  1.54s/it]

Error occurred for model mistral-7b-instruct: Error code: 400 - {'detail': 'Failed to process your request. Please try again later. If you are processing a function, try using a larger model (70b) for better function formatting.'}


international_phonetic_alphabet_transliterate: 100%|██████████| 50/50 [01:27<00:00,  1.74s/it]

BLEU for mixtral-8x22b-instruct: 33.43%
BLEU for mixtral-8x7b-instruct: 5.41%
BLEU for mistral-7b-instruct: 4.51%


### Testing on word_unscrambling

In [ ]:
model_variants = [
    'llama3.2-90b-vision',
    'llama3.2-11b-vision',
    'llama3.2-3b',
    'llama3.2-1b',
    # 'mixtral-8x22b-instruct',
    # 'mixtral-8x7b-instruct',
    # 'mistral-7b-instruct',
]

task_json_path = '/Users/shawna_song/Desktop/598tz_proj/BIG-bench/bigbench/benchmark_tasks/word_unscrambling/task.json'

all_results = {}
for model in model_variants:
    print(f"Evaluating model: {model}")
    all_results[model] = []
    if os.path.isdir(task_path) and os.path.exists(task_json_path):
        with open(task_json_path, 'r') as f:
            bigbench_dataset = json.load(f)
        # print(f"Dataset structure for {task_name}:", json.dumps(bigbench_dataset, indent=2))
        # Evaluate each model variant and store results
        all_results[model] += evaluate_model(model, bigbench_dataset, 200)

Evaluating model: llama3.2-90b-vision


word_unscrambling: 100%|██████████| 200/200 [08:23<00:00,  2.52s/it]


Evaluating model: llama3.2-11b-vision


word_unscrambling: 100%|██████████| 200/200 [04:55<00:00,  1.48s/it]


Evaluating model: llama3.2-3b


...

Error occurred for model mixtral-8x7b-instruct: 'ChatCompletion' object is not subscriptable
Error occurred for model mixtral-8x7b-instruct: 'ChatCompletion' object is not subscriptable
Error occurred for model mixtral-8x7b-instruct: 'ChatCompletion' object is not subscriptable


word_unscrambling:  42%|████▎     | 85/200 [02:47<11:46,  6.14s/it]

Failed to get response after 3 attempts for prompt: The word ionsscide is a scrambled version of the English word 


word_unscrambling:  42%|████▎     | 85/200 [02:48<03:47,  1.98s/it]


KeyboardInterrupt: 

In [99]:
# Print the results for analysis
for model, results in all_results.items():
    print(f"\nResults for {model}:")
    for result in results:
        print(f"Prompt: {result['prompt']}")
        print(f"Generated: {result['generated_text']}")
        print(f"Target: {result['target']}\n")


Results for llama3.2-90b-vision:
Prompt: The word altyuieq is a scrambled version of the English word 
Generated: The word "altyuieq" is a scrambled version of the English word "equality".
Target: equality

Prompt: The word lilybcpu is a scrambled version of the English word 
Generated: The word "lilybcpu" is a scrambled version of the English word "computerbilly" or more likely, "lilybcpu" unscrambled is 'computerbilly' without the extra letters or 'lily' and 'cpu', however, I think it's most likely that the unscrambled version of "lilybcpu" is actually 'bullyclip'.
Target: publicly

Prompt: The word rots is a scrambled version of the English word 
Generated: The word "rots" is a scrambled version of the English word "sort" or also possibly "stork" but most likely it's an anagram of the word "sort".
Target: ['sort', 'otrs']

Prompt: The word alhb is a scrambled version of the English word 
Generated: The word "alhb" is a scrambled version of the English word "BHAL", but I think it's 

In [100]:
def calculate_accuracy(results):
    correct_count = 0
    total_count = len(results)

    for result in results:
        # if result['prompt'] + " " + result['target'] in result['generated_text']:
        #     correct_count += 1
        # elif result['target'] == result['generated_text']:
        #     correct_count += 1
        # print(result)
        if isinstance(result['target'] , list):
            for target_word in result['target']:
                if target_word in result['generated_text']: 
                    correct_count += 1
                    break
        else:
            if result['target'] in result['generated_text']:
                correct_count += 1

    accuracy = (correct_count / total_count) * 100 if total_count > 0 else 0
    return accuracy

for model, results in all_results.items():
    accuracy = calculate_accuracy(results)
    print(f"Accuracy for {model}: {accuracy:.2f}%")

Accuracy for llama3.2-90b-vision: 41.00%
Accuracy for llama3.2-11b-vision: 25.50%
Accuracy for llama3.2-3b: 11.50%
Accuracy for llama3.2-1b: 7.50%
Accuracy for mixtral-8x22b-instruct: 21.50%
Accuracy for mixtral-8x7b-instruct: 0.00%


### Testing onc parsinlu_qa

In [117]:
model_variants = [
    'llama3.2-90b-vision',
    'llama3.2-11b-vision',
    'llama3.2-3b',
    'llama3.2-1b',
    'mixtral-8x22b-instruct',
    'mixtral-8x7b-instruct',
    'mistral-7b-instruct',
]
def evaluate_model(model_name, dataset, num_exp=50):
    # print(f"Evaluating model: {model_name}")
    results = []
    if 'examples' not in dataset:
        raise KeyError("The dataset does not contain the expected 'examples' key.")
    examples = random.sample(dataset['examples'], min(num_exp, len(dataset['examples'])))
    for example in tqdm(examples, desc=f"{dataset['name']}"):
        choices = list(example['target_scores'].keys())
        prompt = example['input'] + " choose from: " + ','.join(choices)
        target = example['target_scores']

        retry_count = 0
        while retry_count < 3:
            try:
                # Generate response using OpenAI client
                response = client.chat.completions.create(
                    model=model_name,
                    messages=[
                        {
                            "role": "system",
                            "content": "Assistant is a large language model trained by OpenAI."
                        },
                        {
                            "role": "user",
                            "content": prompt,
                        }
                    ]
                )
                # print(response)
                try: 
                    generated_text = response.choices[0].message.content.strip()
                except:
                    generated_text = response[0]['message']['content'].strip() if isinstance(response, list) else response['choices'][0]['message']['content'].strip()
                # Store prompt, generated response, and target for analysis
                results.append({
                    'prompt': prompt,
                    'generated_text': generated_text,
                    'target': target
                })
                break
            except Exception as e:
                print(f"Error occurred for model {model_name}: {e}")
                retry_count += 1
                time.sleep(2)  # Wait before retrying
                if retry_count == 3:
                    print(f"Failed to get response after 3 attempts for prompt: {prompt}")
    return results

task_json_path = '/Users/shawna_song/Desktop/598tz_proj/BIG-bench/bigbench/benchmark_tasks/parsinlu_qa/task.json'

all_results = {}
for model in model_variants:
    print(f"Evaluating model: {model}")
    all_results[model] = []
    if os.path.isdir(task_path) and os.path.exists(task_json_path):
        with open(task_json_path, 'r') as f:
            bigbench_dataset = json.load(f)
        # print(f"Dataset structure for {task_name}:", json.dumps(bigbench_dataset, indent=2))
        # Evaluate each model variant and store results
        all_results[model] += evaluate_model(model, bigbench_dataset, 200)

Evaluating model: llama3.2-90b-vision


parsinlu_qa: 100%|██████████| 200/200 [07:25<00:00,  2.23s/it]


Evaluating model: llama3.2-11b-vision


parsinlu_qa: 100%|██████████| 200/200 [04:41<00:00,  1.41s/it]


Evaluating model: llama3.2-3b


...

parsinlu_qa:  52%|█████▎    | 105/200 [03:40<05:47,  3.66s/it]

Error occurred for model mixtral-8x7b-instruct: 'ChatCompletion' object is not subscriptable


parsinlu_qa: 100%|██████████| 200/200 [07:19<00:00,  2.20s/it]


Evaluating model: mistral-7b-instruct


parsinlu_qa: 100%|██████████| 200/200 [06:17<00:00,  1.89s/it]


In [ ]:
# Print the results for analysis
for model, results in all_results.items():
    print(f"\nResults for {model}:")
    for result in results:
        print(f"Prompt: {result['prompt']}")
        print(f"Generated: {result['generated_text']}")
        print(f"Target: {result['target']}\n")

In [118]:
def calculate_accuracy(results):
    correct_count = 0
    total_count = len(results)

    for result in results:
        for choice in result['target'].keys():
            if result['target'][choice] == 1:
                if str(result['target'][choice]) in result['generated_text']:
                    correct_count += 1

    accuracy = (correct_count / total_count) * 100 if total_count > 0 else 0
    return accuracy

for model, results in all_results.items():
    accuracy = calculate_accuracy(results)
    print(f"Accuracy for {model}: {accuracy:.2f}%")

Accuracy for llama3.2-90b-vision: 13.00%
Accuracy for llama3.2-11b-vision: 17.00%
Accuracy for llama3.2-3b: 17.00%
Accuracy for llama3.2-1b: 10.50%
Accuracy for mixtral-8x22b-instruct: 30.00%
Accuracy for mixtral-8x7b-instruct: 36.41%
Accuracy for mistral-7b-instruct: 32.50%
